In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Input, Dense, ReLU
from keras import optimizers, layers
from tensorflow import math
import warnings
warnings.filterwarnings('ignore')

In [2]:
rain = pd.read_csv("./Data/Rainfall.csv")

In [3]:
import glob

slp_path  = "D:/Sahaj/Raw_data/Regrid/slp/*"
sst_path  = "D:/Sahaj/Raw_data/Regrid/sst/*"
# uwnd_path = "./Data/uwnd/*"

slp_months = []
for i in glob.glob(slp_path):
  slp_months.append(i)

sst_months = []
for i in glob.glob(sst_path):
  sst_months.append(i)

# uwnd_months = []
# for i in glob.glob(uwnd_path):
#   uwnd_months.append(i)
  
slp_months.sort()
sst_months.sort()
# uwnd_months.sort()

In [4]:
#Here i am just storing rainfall data of Jun to sep from 1949 to 2014
# index 47= 1948,
import statistics
lpa = statistics.mean(list(rain['Jun-Sep'][60:89]))
print(lpa)
rain_fall_data = list(rain['Jun-Sep'][47:2020])
rain_data = []
for i in range(len(rain_fall_data)):
  temp = (rain_fall_data[i]/lpa)*100
  rain_data.append(temp)

878.0689655172414


In [5]:
from scipy.stats import pearsonr
month_path = slp_months[1] #0th in dex means january
month_data = pd.read_csv(month_path)

In [6]:
# 1948-2000
slp_s,  slp_e  = 0,  53
uwnd_s, uwnd_e = 0,  53
sst_s,  sst_e  = 94, 147
rain_s, rain_e = 47, 100

In [7]:
def get_feature(data):
  new_data = []
  for i in range(len(data)):
    d = data.iloc[i]
    new_data.append(list(d[1:]))
  return np.array(new_data)

# Stacking Data of All Months
combine_slp, combine_sst = [], []
for i in range(12):
  month_path_slp, month_path_sst = slp_months[i], sst_months[i]#, uwnd_months[i] 
  month_data_slp ,month_data_sst = pd.read_csv(month_path_slp), pd.read_csv(month_path_sst)#, pd.read_csv(month_path_uwnd)
 
  feature_slp, feature_sst = get_feature(month_data_slp), get_feature(month_data_sst)#, get_feature(month_data_uwnd)
  feature_slp, feature_sst = feature_slp[slp_s:slp_e], feature_sst[sst_s:sst_e]#, feature_uwnd[uwnd_s:uwnd_e]
  for i in feature_slp:
    combine_slp.append(i)
  for i in feature_sst:
    combine_sst.append(i)
  # for i in feature_uwnd:
  #   combine_uwnd.append(i)
x_slp = np.array(combine_slp)
x_sst = np.array(combine_sst)
# x_uwnd = np.array(combine_uwnd)
x = np.concatenate((x_slp, x_sst), axis=1)
x.shape, x_slp.shape, x_sst.shape

((636, 389), (636, 324), (636, 65))

In [8]:
def get_lr(initial_learning_rate=0.001, decay_steps=4000, decay_rate=1):
    return optimizers.schedules.ExponentialDecay(
            initial_learning_rate,
            decay_steps=decay_steps,
            decay_rate=decay_rate,
            staircase=True)

In [9]:
def get_model_1(n_inputs=389):
  visible = Input(shape=(n_inputs,))
  e = Dense(n_inputs, kernel_regularizer=keras.regularizers.L2(0.01))(visible)
  e = ReLU()(e)
  bottleneck = Dense(97, kernel_regularizer=keras.regularizers.L2(0.01))(e)
  output = Dense(n_inputs, activation='linear', kernel_regularizer=keras.regularizers.L2(0.01))(bottleneck)
  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer='adam', loss='mse')
  return model

In [10]:
model_1 = get_model_1()
model_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 389)]             0         
                                                                 
 dense (Dense)               (None, 389)               151710    
                                                                 
 re_lu (ReLU)                (None, 389)               0         
                                                                 
 dense_1 (Dense)             (None, 97)                37830     
                                                                 
 dense_2 (Dense)             (None, 389)               38122     
                                                                 
Total params: 227,662
Trainable params: 227,662
Non-trainable params: 0
_________________________________________________________________


In [11]:
history = model_1.fit(x, x, epochs=8000, batch_size=16, verbose=2, validation_data=(x,x))

Epoch 1/8000
40/40 - 1s - loss: 8689.2725 - val_loss: 5467.8447 - 717ms/epoch - 18ms/step
Epoch 2/8000
40/40 - 0s - loss: 4550.4888 - val_loss: 3471.8711 - 118ms/epoch - 3ms/step
Epoch 3/8000
40/40 - 0s - loss: 3003.6355 - val_loss: 2413.6140 - 120ms/epoch - 3ms/step
Epoch 4/8000
40/40 - 0s - loss: 2221.5693 - val_loss: 1876.2889 - 123ms/epoch - 3ms/step
Epoch 5/8000
40/40 - 0s - loss: 1776.8785 - val_loss: 1533.8134 - 124ms/epoch - 3ms/step
Epoch 6/8000
40/40 - 0s - loss: 1486.1660 - val_loss: 1303.8290 - 124ms/epoch - 3ms/step
Epoch 7/8000
40/40 - 0s - loss: 1271.5693 - val_loss: 1135.9987 - 137ms/epoch - 3ms/step
Epoch 8/8000
40/40 - 0s - loss: 1127.5376 - val_loss: 1014.6105 - 143ms/epoch - 4ms/step
Epoch 9/8000
40/40 - 0s - loss: 1006.9927 - val_loss: 915.2680 - 142ms/epoch - 4ms/step
Epoch 10/8000
40/40 - 0s - loss: 917.1693 - val_loss: 835.4125 - 146ms/epoch - 4ms/step
Epoch 11/8000
40/40 - 0s - loss: 849.8038 - val_loss: 774.7538 - 142ms/epoch - 4ms/step
Epoch 12/8000
40/40 - 0

In [12]:
model_temp1 = Model(inputs=model_1.input, outputs=model_1.layers[3].output)
model_2_inputs = model_temp1(x)
model_2_inputs.shape

TensorShape([636, 97])

In [13]:
def get_model_2(n_inputs=97):
  visible = Input(shape=(n_inputs,))
  e = Dense(n_inputs, kernel_regularizer=keras.regularizers.L2(0.01))(visible)
  e = ReLU()(e)
  bottleneck = Dense(29, kernel_regularizer=keras.regularizers.L2(0.01))(e)
  output = Dense(n_inputs, activation='linear', kernel_regularizer=keras.regularizers.L2(0.01))(bottleneck)
  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer='adam', loss='mse')
  return model

In [14]:
model_2 = get_model_2()
model_2.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 97)]              0         
                                                                 
 dense_3 (Dense)             (None, 97)                9506      
                                                                 
 re_lu_1 (ReLU)              (None, 97)                0         
                                                                 
 dense_4 (Dense)             (None, 29)                2842      
                                                                 
 dense_5 (Dense)             (None, 97)                2910      
                                                                 
Total params: 15,258
Trainable params: 15,258
Non-trainable params: 0
_________________________________________________________________


In [15]:
history = model_2.fit(model_2_inputs, model_2_inputs, epochs=8000, batch_size=16, verbose=2, validation_data=(model_2_inputs,model_2_inputs))

Epoch 1/8000
40/40 - 2s - loss: 9292.2051 - val_loss: 7915.6279 - 2s/epoch - 38ms/step
Epoch 2/8000
40/40 - 0s - loss: 7354.9048 - val_loss: 6671.4458 - 102ms/epoch - 3ms/step
Epoch 3/8000
40/40 - 0s - loss: 6188.9946 - val_loss: 5582.7700 - 153ms/epoch - 4ms/step
Epoch 4/8000
40/40 - 0s - loss: 5288.4082 - val_loss: 4941.6353 - 129ms/epoch - 3ms/step
Epoch 5/8000
40/40 - 0s - loss: 4791.7988 - val_loss: 4537.4810 - 195ms/epoch - 5ms/step
Epoch 6/8000
40/40 - 0s - loss: 4416.0269 - val_loss: 4195.1587 - 154ms/epoch - 4ms/step
Epoch 7/8000
40/40 - 0s - loss: 4098.8784 - val_loss: 3901.3569 - 211ms/epoch - 5ms/step
Epoch 8/8000
40/40 - 0s - loss: 3826.7373 - val_loss: 3650.5078 - 178ms/epoch - 4ms/step
Epoch 9/8000
40/40 - 0s - loss: 3597.2688 - val_loss: 3442.5181 - 202ms/epoch - 5ms/step
Epoch 10/8000
40/40 - 0s - loss: 3404.7810 - val_loss: 3271.8384 - 110ms/epoch - 3ms/step
Epoch 11/8000
40/40 - 0s - loss: 3246.6113 - val_loss: 3122.7490 - 204ms/epoch - 5ms/step
Epoch 12/8000
40/40 -

In [16]:
model_temp2 = Model(inputs=model_2.input, outputs=model_2.layers[3].output)
model_3_inputs = model_temp2(model_2_inputs)
model_3_inputs.shape

TensorShape([636, 29])

In [17]:
def get_model_3(n_inputs=29):
  visible = Input(shape=(n_inputs,))
  e = Dense(n_inputs, kernel_regularizer=keras.regularizers.L2(0.01))(visible)
  e = ReLU()(e)
  bottleneck = Dense(9, kernel_regularizer=keras.regularizers.L2(0.01))(e)
  output = Dense(n_inputs, activation='linear', kernel_regularizer=keras.regularizers.L2(0.01))(bottleneck)
  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')
  return model

In [18]:
model_3 = get_model_3()
model_3.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 29)]              0         
                                                                 
 dense_6 (Dense)             (None, 29)                870       
                                                                 
 re_lu_2 (ReLU)              (None, 29)                0         
                                                                 
 dense_7 (Dense)             (None, 9)                 270       
                                                                 
 dense_8 (Dense)             (None, 29)                290       
                                                                 
Total params: 1,430
Trainable params: 1,430
Non-trainable params: 0
_________________________________________________________________


In [19]:
history = model_3.fit(model_3_inputs, model_3_inputs, epochs=8000, batch_size=16, verbose=2, validation_data=(model_3_inputs,model_3_inputs))

Epoch 1/8000
40/40 - 0s - loss: 2295.9106 - val_loss: 2058.0000 - 491ms/epoch - 12ms/step
Epoch 2/8000
40/40 - 0s - loss: 1958.2681 - val_loss: 1860.2805 - 63ms/epoch - 2ms/step
Epoch 3/8000
40/40 - 0s - loss: 1802.6140 - val_loss: 1738.3730 - 51ms/epoch - 1ms/step
Epoch 4/8000
40/40 - 0s - loss: 1695.3599 - val_loss: 1644.1091 - 68ms/epoch - 2ms/step
Epoch 5/8000
40/40 - 0s - loss: 1608.0461 - val_loss: 1563.3309 - 53ms/epoch - 1ms/step
Epoch 6/8000
40/40 - 0s - loss: 1532.2147 - val_loss: 1492.8850 - 69ms/epoch - 2ms/step
Epoch 7/8000
40/40 - 0s - loss: 1466.0450 - val_loss: 1431.6873 - 62ms/epoch - 2ms/step
Epoch 8/8000
40/40 - 0s - loss: 1408.9302 - val_loss: 1378.7216 - 62ms/epoch - 2ms/step
Epoch 9/8000
40/40 - 0s - loss: 1359.2883 - val_loss: 1332.7273 - 48ms/epoch - 1ms/step
Epoch 10/8000
40/40 - 0s - loss: 1316.4740 - val_loss: 1293.1998 - 75ms/epoch - 2ms/step
Epoch 11/8000
40/40 - 0s - loss: 1280.3979 - val_loss: 1260.5935 - 64ms/epoch - 2ms/step
Epoch 12/8000
40/40 - 0s - l

In [20]:
def get_model_final(n_inputs=389):
  visible = Input(shape=(n_inputs,))
  e = Dense(n_inputs, kernel_regularizer=keras.regularizers.L2(0.01))(visible)
  e = ReLU()(e)
  e = Dense(97, kernel_regularizer=keras.regularizers.L2(0.01))(e)
  e = ReLU()(e)
  e = Dense(29, kernel_regularizer=keras.regularizers.L2(0.01))(e)
  e = ReLU()(e)
  bottleneck = Dense(9, kernel_regularizer=keras.regularizers.L2(0.01))(e)
  e = Dense(29, kernel_regularizer=keras.regularizers.L2(0.01))(bottleneck)
  e = ReLU()(e)
  e = Dense(97, kernel_regularizer=keras.regularizers.L2(0.01))(e)
  e = ReLU()(e)
  output = Dense(n_inputs, activation='linear', kernel_regularizer=keras.regularizers.L2(0.01))(e)
  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer='adam', loss='mse')
  return model

In [21]:
model_final = get_model_final()
model_final.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 389)]             0         
                                                                 
 dense_9 (Dense)             (None, 389)               151710    
                                                                 
 re_lu_3 (ReLU)              (None, 389)               0         
                                                                 
 dense_10 (Dense)            (None, 97)                37830     
                                                                 
 re_lu_4 (ReLU)              (None, 97)                0         
                                                                 
 dense_11 (Dense)            (None, 29)                2842      
                                                                 
 re_lu_5 (ReLU)              (None, 29)                0   

In [22]:
model_final.layers[2] = model_1.layers[3]
model_final.layers[11] = model_1.layers[4]
model_final.layers[4] = model_2.layers[3]
model_final.layers[9] = model_2.layers[4]
model_final.layers[6] = model_3.layers[3]
model_final.layers[7] = model_3.layers[4]
model_final.compile(optimizer='adam', loss='mse')

In [23]:
model_final.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 389)]             0         
                                                                 
 dense_9 (Dense)             (None, 389)               151710    
                                                                 
 re_lu_3 (ReLU)              (None, 389)               0         
                                                                 
 dense_10 (Dense)            (None, 97)                37830     
                                                                 
 re_lu_4 (ReLU)              (None, 97)                0         
                                                                 
 dense_11 (Dense)            (None, 29)                2842      
                                                                 
 re_lu_5 (ReLU)              (None, 29)                0   

In [24]:
history = model_final.fit(x, x, epochs=10000, batch_size=16, verbose=2, validation_data=(x,x))

Epoch 1/10000
40/40 - 1s - loss: 11077.0371 - val_loss: 8436.3760 - 974ms/epoch - 24ms/step
Epoch 2/10000
40/40 - 0s - loss: 7344.8945 - val_loss: 6540.0371 - 161ms/epoch - 4ms/step
Epoch 3/10000
40/40 - 0s - loss: 6481.0620 - val_loss: 6175.2568 - 157ms/epoch - 4ms/step
Epoch 4/10000
40/40 - 0s - loss: 6054.8604 - val_loss: 5773.2021 - 163ms/epoch - 4ms/step
Epoch 5/10000
40/40 - 0s - loss: 5646.6694 - val_loss: 5375.9204 - 162ms/epoch - 4ms/step
Epoch 6/10000
40/40 - 0s - loss: 5313.9443 - val_loss: 5074.7759 - 153ms/epoch - 4ms/step
Epoch 7/10000
40/40 - 0s - loss: 5073.0039 - val_loss: 4935.3052 - 124ms/epoch - 3ms/step
Epoch 8/10000
40/40 - 0s - loss: 4887.2393 - val_loss: 4742.2324 - 141ms/epoch - 4ms/step
Epoch 9/10000
40/40 - 0s - loss: 4727.8496 - val_loss: 4590.3179 - 140ms/epoch - 4ms/step
Epoch 10/10000
40/40 - 0s - loss: 4582.3115 - val_loss: 4460.1948 - 135ms/epoch - 3ms/step
Epoch 11/10000
40/40 - 0s - loss: 4415.0391 - val_loss: 4261.9741 - 160ms/epoch - 4ms/step
Epoch 

In [25]:
model_final.save("./Data/Models/model_final_slp_sst.h5")
# model_final = keras.models.load_model('./Data/Models/model_final.h5')

In [26]:
class Custom_Layer(layers.Layer):
    def __init__(self, weights, bias=False):
        super(Custom_Layer, self).__init__()
        self.w = weights
        if bias:
            self.b = bias

    def call(self, inputs):
        try: return tf.matmul(inputs, self.w) + self.b
        except: return tf.matmul(inputs, self.w)

In [27]:
std = []
for i in model_final.layers:
    # print(i.weights[0])
    try: std.append(np.array(math.reduce_std(i.weights[0])) * 2)
    except: std.append([])

In [28]:
final_layers = keras.Sequential([Input(shape=(None, 389))])
for i, layer in enumerate(model_final.layers):
    try:
        temp_layer = []
        for ind, j in enumerate(layer.weights[0]):
            temp_weights = []
            for index, k in enumerate(j):
                if k < std[i]: temp_weights.append(0)
                else:          temp_weights.append(1)  
            temp_layer.append(temp_weights)
        weights = tf.Variable(temp_layer, dtype='float32')
        try: final_layers.add(Custom_Layer(weights, bias=layer.weights[1]))
        except: final_layers.add(Custom_Layer(weights))
    except IndexError: final_layers.add(layer)

In [29]:
final_layers.compile(optimizer='adam', loss='mse')
final_layers.build(input_shape=(None, 389))
final_layers.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        multiple                  0         
                                                                 
 custom__layer_1 (Custom_Lay  (None, None, 389)        151321    
 er)                                                             
                                                                 
 re_lu_3 (ReLU)              multiple                  0         
                                                                 
 custom__layer_3 (Custom_Lay  (None, None, 97)         37733     
 er)                                                             
                                                                 
 re_lu_4 (ReLU)              multiple                  0         
                                                                 
 custom__layer_5 (Custom_Lay  (None, None, 29)         2

In [30]:
model_final1 = Model(inputs=model_final.input, outputs=model_final.layers[3].output)
model_final1.trainable = False

model_final2 = Model(inputs=model_final.input, outputs=model_final.layers[5].output)
model_final2.trainable = False

model_final3 = Model(inputs=model_final.input, outputs=model_final.layers[7].output)
model_final3.trainable = False

models = [model_final1, model_final2, model_final3]
# models = [model_final1, model_final4]

Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: KerasTensor(type_spec=TensorSpec(shape=(None, 389), dtype=tf.float32, name='input_4'), name='input_4', description="created by layer 'input_4'")
Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: KerasTensor(type_spec=TensorSpec(shape=(None, 389), dtype=tf.float32, name='input_4'), name='input_4', description="created by layer 'input_4'")
Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: KerasTensor(type_spec=TensorSpec(shape=(None, 389), dtype=tf.float32, name='input_4'), name='input_4', description="created by layer 'input_4'")


In [31]:
final_layers1 = Model(inputs=final_layers.input, outputs=final_layers.layers[3].output)
final_layers1.trainable = False

final_layers2 = Model(inputs=final_layers.input, outputs=final_layers.layers[5].output)
final_layers2.trainable = False

final_layers3 = Model(inputs=final_layers.input, outputs=final_layers.layers[7].output)
final_layers3.trainable = False

models = [final_layers1, final_layers2, final_layers3]

In [32]:
def get_feature(data):
  new_data = []
  for i in range(len(data)):
    d = data.iloc[i]
    new_data.append(list(d[1:]))
  # print(new_data)
  return np.array(new_data)

# to combine all the features together
def predictor(month,mod):
  # chacking for each MONTH
  month_path_slp, month_path_sst = slp_months[i], sst_months[i]
  # reading month data
  month_data_slp ,month_data_sst = pd.read_csv(month_path_slp), pd.read_csv(month_path_sst)
  feature_slp, feature_sst = get_feature(month_data_slp), get_feature(month_data_sst)
  feature_slp, feature_sst = feature_slp[slp_s:slp_e], feature_sst[sst_s:sst_e]
  x_slp = np.array(combine_slp)
  x_sst = np.array(combine_sst)
  feature = np.concatenate((x_slp, x_sst), axis=1)
  pred = np.array(feature)
  try: pred_m = models[mod](feature)
  except: 
    print(i)
    raise(IndexError)
  pred_f = pred_m.numpy()
  return pred_f

In [33]:
from sklearn.svm import SVR


#w_start = window start, w_end = window end  
def get_top_pred(top,pred_info,w_start,w_end):
  k = 0
  pred_f = pred_info
  cor_list = []
  for i in range(len(pred_f[0])):
    score = []
    
    for j in range(w_start,w_end):
      score.append(pred_f[j][i])
    corr, _ = pearsonr(rain_fall_data[w_start:w_end],score)
    cor_list.append(corr)
    # print(corr)
  list1=list(enumerate(cor_list)) #I have taken cosine rank[0] beacuse it is inside the list
  list2=sorted(list1, key=lambda x: x[1],reverse=True)
  top_feature_index = []
  #here i am getting the top features index
  for i in range(top):
    index = list2[i][0]
    top_feature_index.append(index)
  # Here i am getting all the top features
  predictor = []
  for i in range(len(pred_f)):
    temp = []
    for j in top_feature_index:
      # print(i,"  ",j)
      feature = pred_f[i][j]
      temp.append(feature)
    predictor.append(temp)
  # print(list2)
  return predictor
  # print(list1)


# np.concatenate((a, b.T), axis=1)
def get_predictors(months,top,models,w_start,w_end):
  for i in months:
    for mod in range(len(models)):
      y_pred = predictor(i,mod)
      b = get_top_pred(top, y_pred, w_start,w_end)
      try:    pred = np.concatenate((pred,b), axis=1)
      except: pred = np.array(b)
  return pred

def window_solution(months,top):
  reg = SVR(kernel = 'rbf',C=1.0,epsilon=0.45)
  # reg = make_pipeline(StandardScaler(), SVR(kernel = 'rbf',C=1.0, epsilon=0.2))
  # months = [0]
  k=0
  cor_all = []
  for window in range(10,53):
    k+=1
    score = []
    pred = get_predictors(months,top,models,53-window,53)
    for i in range(14):
      start = 53-window
      end = 53+1
      # pred = get_predictors(months,2,models,start,end)
      reg.fit(pred[53-window:53+i], rain_fall_data[53-window:53+i])
      score.append(reg.predict([pred[53+i]])[0])
    # print(score)
    corr, _ = pearsonr(rain_fall_data[53:67],score)
    print("Window size = ",window,"   plcc",corr)
    cor_all.append(corr)
  return min(cor_all),max(cor_all)
#This is by taking correlation after every iteraion of each predict

In [34]:
from itertools import combinations
# months_comb = [j for i in range(4) for j in combinations(range(5,9), i+1)]
months_comb = [[4]]
top = 9
# models = [model_final1, model_final2, model_final3]
models = [final_layers1, final_layers2, final_layers3]

minimum, maximum = 1, -1
for month in months_comb:
  for i in range(1,top):
    left, right = window_solution(month,i)
    print("\n\nMonth combination = ",month,"    top = ",i,"   min and max",left,"   ",right)
    if minimum > left:  minimum, min_month, min_top = left, month, i
    if maximum < right: maximum, max_month, max_top = right, month, i
    print(f'Minimum: {minimum}, {min_month}, {min_top}    Maximum: {maximum}, {max_month}, {max_top}\n')

Window size =  10    plcc -0.29950230388260296
Window size =  11    plcc -0.29085987905054805
Window size =  12    plcc -0.34354791206730034
Window size =  13    plcc -0.30427682625190944
Window size =  14    plcc -0.3219754019022026
Window size =  15    plcc -0.28853212823294505
Window size =  16    plcc -0.19138341653751545
Window size =  17    plcc -0.28782210572526534
Window size =  18    plcc -0.3180984146883392
Window size =  19    plcc -0.2909295736034011
Window size =  20    plcc -0.29159926458193053
Window size =  21    plcc -0.10285017160288698
Window size =  22    plcc -0.287458270889252
Window size =  23    plcc -0.10783431362890011
Window size =  24    plcc -0.17302004886599343
Window size =  25    plcc -0.25577600709533904
Window size =  26    plcc -0.2829024716841418
Window size =  27    plcc -0.2531285385276982
Window size =  28    plcc -0.2879511095504062
Window size =  29    plcc -0.262492265579744
Window size =  30    plcc -0.2671512291870978
Window size =  31    plc